In [1]:
import sys
print(sys.path)
sys.path.append("../KE/")
print(sys.path)

['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Library/Caches/pypoetry/virtualenvs/kg-microbe-pu1PEBVS-py3.10/lib/python3.10/site-packages', '/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe']
['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Library/Caches/pypoetry/virtualenvs/kg-microbe-pu1PEBVS-py3.10/lib/python3.10/site-packages', '/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe', '../KE/']


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

from KE.classifier.prepare import split_dataset
from KE.dataclean import tables

In [3]:
data = pd.read_csv("../data/merged/merged-kg_edges.tsv", header=0, sep="\t")

In [4]:
data

,id,subject,predicate,object,relation,provided_by,knowledge_source
0,urn:uuid:c672b249-cb5c-4ed6-b6f6-944bbf690d10,NCBITaxon:10,biolink:subclass_of,NCBITaxon:1706371,rdfs:subClassOf,NaN,Graph
1,urn:uuid:a9b6d704-f957-483d-a02f-5b0dda8b2aca,NCBITaxon:100,biolink:subclass_of,NCBITaxon:99,rdfs:subClassOf,NaN,Graph
2,urn:uuid:dcf90aaa-34fb-494c-afb8-dd30286457be,NCBITaxon:100,biolink:occurs_in,mediadive.medium:7,BAO:0002924,NaN,Graph
3,urn:uuid:94f4ef9b-867e-4300-a720-2d5d7ca54230,NCBITaxon:100,biolink:capable_of,traits.pathways:hydrogen_oxidation_dark,RO:0002215,NaN,Graph
4,urn:uuid:e69ce7ed-ac8e-4d34-9657-d511661af66d,NCBITaxon:100,biolink:capable_of,traits.pathways:aerobic_chemo_heterotrophy,RO:0002215,NaN,Graph
...,...,...,...,...,...,...,...
1324132,urn:uuid:34043e25-2e43-4e5d-a156-030b7803cb33,NCBITaxon:147448,biolink:capable_of,ECOCORE:00000177,RO:0002215,NaN,Graph
1324133,urn:uuid:10747a51-1c4e-4b89-aa75-7082fc53f7ca,NCBITaxon:147448,biolink:has_phenotype,traits.cell_shape_enum:coccus,RO:0002200,NaN,Graph
1324134,urn:uuid:25ebadb1-3ba3-424d-b97c-ccc4214afebe,NCBITaxon:1240726,biolink:has_phenotype,traits.cell_shape_enum:bacillus,RO:0002200,NaN,Graph
1324135,urn:uuid:18331d56-f89d-40bd-90c0-085ca5209a1c,NCBITaxon:1240783,biolink:has_phenotype,traits.cell_shape_enum:bacillus,RO:0002200,NaN,Graph


In [5]:
data_pairs = data[['subject','object']].drop_duplicates()
data_pairs

,subject,object
0,NCBITaxon:10,NCBITaxon:1706371
1,NCBITaxon:100,NCBITaxon:99
2,NCBITaxon:100,mediadive.medium:7
3,NCBITaxon:100,traits.pathways:hydrogen_oxidation_dark
4,NCBITaxon:100,traits.pathways:aerobic_chemo_heterotrophy
...,...,...
1324132,NCBITaxon:147448,ECOCORE:00000177
1324133,NCBITaxon:147448,traits.cell_shape_enum:coccus
1324134,NCBITaxon:1240726,traits.cell_shape_enum:bacillus
1324135,NCBITaxon:1240783,traits.cell_shape_enum:bacillus


In [6]:
#data_pairs_clean = tables.rows_and_cols_quant_filter(data_pairs)
data_pairs_clean = data_pairs.groupby('object').filter(lambda x : len(x)>9)

In [7]:
data_pairs_clean

,subject,object
0,NCBITaxon:10,NCBITaxon:1706371
1,NCBITaxon:100,NCBITaxon:99
3,NCBITaxon:100,traits.pathways:hydrogen_oxidation_dark
4,NCBITaxon:100,traits.pathways:aerobic_chemo_heterotrophy
5,NCBITaxon:100,traits.pathways:thiosulfate_oxidation_dark
...,...,...
1324132,NCBITaxon:147448,ECOCORE:00000177
1324133,NCBITaxon:147448,traits.cell_shape_enum:coccus
1324134,NCBITaxon:1240726,traits.cell_shape_enum:bacillus
1324135,NCBITaxon:1240783,traits.cell_shape_enum:bacillus


In [8]:
# Splitting the data into features and target labels
X = data_pairs_clean[['subject']]
y = data_pairs_clean['object']

# Convert the input labels column to a categorical type if it isn't
X['subject'] = X['subject'].astype('category')

# Convert categorical columns to integers
X['subject'] = X['subject'].cat.codes

/var/folders/hx/svkm0dlj0bd9689lm21_7fyw0000gs/T/ipykernel_28797/29537296.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['subject'] = X['subject'].astype('category')
/var/folders/hx/svkm0dlj0bd9689lm21_7fyw0000gs/T/ipykernel_28797/29537296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['subject'] = X['subject'].cat.codes


In [9]:
## Train the model
#train_data = Pool(data=X, label=y, cat_features=[0])
#model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
#model.fit(train_data)



In [10]:
## Predict on new data
#new_data = pd.DataFrame({'subject': ['A', 'C']})
#new_data['subject'] = new_data['subject'].astype('category').cat.codes
#predictions = model.predict(new_data)
#
#print(predictions)

In [11]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the model
train_data = Pool(data=X_train, label=y_train, cat_features=[0])
test_data = Pool(data=X_test, label=y_test, cat_features=[0])


In [ ]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(train_data)#, plot=True)

In [39]:
# Predict on test data
y_pred = model.predict(test_data)
y_pred_proba = model.predict_proba(test_data)[:,1]  # Probabilities for the positive class

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))

Accuracy: 0.1955521472392638

Classification Report:
               precision    recall  f1-score   support

      DSMZ:1       0.45      0.22      0.30        76
    DSMZ:104       0.17      0.03      0.05        34
   DSMZ:1040       0.00      0.00      0.00         4
   DSMZ:104b       0.00      0.00      0.00        16
   DSMZ:104c       0.00      0.00      0.00         6
  DSMZ:1076b       0.00      0.00      0.00        11
     DSMZ:11       0.25      0.04      0.06        28
    DSMZ:110       0.00      0.00      0.00        13
   DSMZ:110a       0.00      0.00      0.00         5
   DSMZ:1113       0.00      0.00      0.00         3
   DSMZ:1122       0.00      0.00      0.00         4
   DSMZ:1166       0.00      0.00      0.00         4
    DSMZ:119       0.00      0.00      0.00         3
   DSMZ:1193       0.00      0.00      0.00         1
  DSMZ:1203a       0.00      0.00      0.00         8
    DSMZ:141       0.00      0.00      0.00         3
   DSMZ:141c       0.00    

/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

ValueError: multi_class must be in ('ovo', 'ovr')